In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path

from IPython.display import display
from lib.util_plot import *
from lib.constants import *

import lib.VIS_L23_preprocessing.vis_L23_constants as VIS
from lib.pandas_impl import *
from lib.pandas_stats_impl import *
from lib.pandas_stats_VIS import VISAggregateStatistics
from lib.multilevel_analysis import MultilevelAnalysis
from models import *

### Prepare data

In [ ]:
data_folder = Path.cwd() / 'data' / 'VIS'
eval_folder = Path.cwd() / 'data' / 'eval' / 'VIS_24-12-14_25mu'
eval_folder.mkdir(parents=True, exist_ok=True)
plot_folder = eval_folder / "plots"
plot_folder.mkdir(parents=True, exist_ok=True)
synapse_file = data_folder / 'synapses_grid-25000_aggregated.csv'

In [ ]:
def merge_inh_celltypes(df_summary):
    df = df_summary.copy()

    if("post_celltype_merged" not in df.index.names):
        post_celltype_merged = df.index.get_level_values("post_celltype").values.copy()
        post_celltype_merged[post_celltype_merged > 1] = 2 
        df.loc[:, "post_celltype_merged"] = post_celltype_merged
        df.set_index("post_celltype_merged", append=True, inplace=True)

    if("pre_celltype_merged" not in df.index.names):
        pre_celltype_merged = df.index.get_level_values("pre_celltype").values.copy()
        pre_celltype_merged[pre_celltype_merged > 1] = 2 
        df.loc[:, "pre_celltype_merged"] = pre_celltype_merged
        df.set_index("pre_celltype_merged", append=True, inplace=True)

    df.sort_index(inplace=True)
    return df

In [ ]:
df_synapses = pd.read_csv(synapse_file)

# filter self connections
df_synapses = df_synapses[(df_synapses.pre_id_mapped == -1) | (df_synapses.pre_id_mapped != df_synapses.post_id_mapped)]
df_synapses.reset_index(drop=True, inplace=True)

df_synapses_indexed = df_synapses.set_index(["pre_celltype", "post_celltype",  \
                                             "pre_id_mapped", "post_id_mapped", "post_compartment", "overlap_volume"])

df_synapses_indexed = merge_inh_celltypes(df_synapses_indexed)

df_synapses_indexed.sort_index(inplace=True)
df_synapses_indexed.synapse_count.sum()

Load or initialize local overlap volumes and global index data

In [ ]:
index_data = compile_index_data(eval_folder, df_synapses_indexed)

### Model definition

In [ ]:
model_P_impl = Model(index_data, ["pre_celltype_merged", "post_celltype_merged"])
model_P_disaggregated_impl = Model(index_data, ["pre_celltype", "post_celltype"])
model_S_impl = Model(index_data, ["post_compartment"])

# cellular-resolution models 
model_C_impl = Model(index_data, ["pre_id_mapped", "post_id_mapped"])
model_Ca_impl = Model(index_data, ["pre_id_mapped"])
model_Cb_impl = Model(index_data, ["post_id_mapped"])

model_Pa_impl = Model(index_data, ["post_celltype", "pre_id_mapped"])
model_PSa_impl = Model(index_data, ["post_celltype", "post_compartment", "pre_id_mapped"])
model_Sb_impl = Model(index_data, ["post_compartment", "post_id_mapped"])

### Run models

#### Group-level models

In [ ]:
statistics = VISAggregateStatistics(index_data, compute_syncounts=True, compute_motifs=True, compute_clusters=True)
multilevel_analysis = MultilevelAnalysis(index_data, df_synapses_indexed, statistics, num_realizations=100)
multilevel_analysis.run_null_and_empirical()

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_P_impl,
    MODEL_P
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_P_disaggregated_impl,
    MODEL_P_disaggregated
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_P,
    model_S_impl,
    MODEL_PS
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_P_disaggregated,
    model_S_impl,
    MODEL_PS_disaggregated
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS_disaggregated,
    model_Ca_impl,
    MODEL_PSCa
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS_disaggregated,
    model_Cb_impl,
    MODEL_PSCb
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_Pa_impl,
    MODEL_Pa
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_Pa,
    model_S_impl,
    MODEL_PaS
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_Pa,
    model_Sb_impl,
    MODEL_PaSb
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_PSa_impl,
    MODEL_PSa
)

#### Cellular-level models

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_C_impl,
    MODEL_C
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS_disaggregated,
    model_Ca_impl,
    MODEL_PSCa
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_PS_disaggregated,
    model_Cb_impl,
    MODEL_PSCb
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_Pa_impl,
    MODEL_Pa
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_NULL,
    model_PSa_impl,
    MODEL_PSa
)

In [ ]:
multilevel_analysis.run_model(
    MODEL_Pa,
    model_Sb_impl,
    MODEL_PaSb
)

#### Save results

In [ ]:
filename = eval_folder / "multilevel_analysis.pkl"
with open(filename, 'wb') as file:
    pickle.dump(multilevel_analysis, file)
